# Parte 3: Recolección de información
### Web Scraping a "Semana" para investigación referente a los cambios climáticos en Colombia

### Paso 1: Importación de librerias

In [1]:
import pandas as pd
import time
import random
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from random import shuffle

### Paso 2: Definición de opciones para el navegador

In [2]:
chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument('--window-size=1420,1080')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument("--disable-notifications")
chrome_options.add_argument("--remote-debugging-port=9222")
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)
chrome_options.add_experimental_option("excludeSwitches", ["disable-popup-blocking"])

### Paso 3: Definición de clase personalizada

In [3]:
# Clase para personalización
class bcolors:
    WARNING = '\033[93m'
    FAIL = '\033[91m'

# Control de errores
try:   
    # Definición de función
    def Seconds(result):
        hours = round(result/3600)
        minutes = round((result - (hours * 3600))/60)
        seconds = round(result - ((hours * 3600) + (minutes * 60)))
        if seconds < 0: seconds = seconds*-1
        if minutes < 0: minutes = minutes*-1
        final_time = "El sitema tardó: " + str(hours) + " horas " + str(minutes) + " minutos " + str(seconds) + " segundos en finalizar su ejeción"
        return final_time
    
except Exception as e: 
    print(bcolors.FAIL + "Revisa el error => ", e) # Impresión del error personalizado

### Paso 4: Lectura de archivo de noticias

In [4]:
# Definición de la clase
class IterableClass:
    
    # Función inicial o constructor
    def __init__(self, list_values = None): # None es para que sea opcional
        self.list_values = list_values
        self.dateframe = pd.read_csv(r'df_clean_1.csv') # lectura del archivo
        self.i = 0
        
    # Retornar un iterador
    def __iter__(self):
        return self
    
    # Retorna un elemento del contenedor
    def __next__(self):
        if self.i >= len(self.list_values):
            print("\nNo hay más valores a recorrer... ⬇")
            raise StopIteration
            
        item = self.list_values[self.i]
        self.i += 1
        return item  

### Paso 5: Instacia del objeto e impresión del DataFrame

In [5]:
# Instancia del objeto
IC = IterableClass()

# 5.1. Eliminación de columna que se creo en por error "Unnamed: 0" .drop('Unnamed: 0', axis=1)
# 5.2. Reseteo de index por la acción de combinación de DataFrames .reset_index().drop(['index'], axis=1)
IC.dateframe.drop('Unnamed: 0', axis=1).reset_index().drop(['index'], axis=1)

,headers_text,headers_link
0,África abre sus puertas a inversiones verdes e...,https://www.semana.com/mundo/africa/articulo/a...
1,Inundaciones en Brasil: un testimonio impactan...,https://www.semana.com/mundo/articulo/inundaci...
2,Responsable climático de la ONU critica fuerte...,https://www.semana.com/mundo/articulo/responsa...
3,"“El cambio climático es real y es aterrador”, ...",https://www.semana.com/cultura/cine/articulo/e...
4,“Los últimos ocho años fueron los más caluroso...,https://www.semana.com/sostenible/medio-ambien...
...,...,...
295,"Océanos calientes, el combustible para los hur...",https://www.semana.com/medio-ambiente/articulo...
296,Crece la amenaza de conflictos por el agua en ...,https://www.semana.com/crece-la-amenaza-de-con...
297,La mitad de las playas del mundo podrían desap...,https://www.semana.com/la-mitad-de-las-playas-...
298,Trancón de delfines rosados por bajo caudal en...,https://www.semana.com/trancon-de-delfines-ros...


### Paso 6: Definición de función para unificar los diferentes parrafos de la noticia

In [6]:
def JoinParagraphs(paragraphs):
    TextParagraphs = ""
    for paragraph in paragraphs:
        TextParagraphs = TextParagraphs + paragraph.text
        
    return TextParagraphs

### Paso 7: Definición de función para unificar los diferentes temas de la noticia

In [7]:
def JoinThemes(themes):
    TextThemes = ""
    for theme in themes:
        TextThemes = TextThemes + theme.text + ", "
    return TextThemes

### Paso 8: Recolección de información para cada noticia

In [8]:
# 8.1 Declaraciones de variables
titles = []
summaries = []
authors = []
dates = []
paragraphs = []
themes = []

In [9]:
# PASO 8.2: Aviso para el usuario final de que se inicio el web-scraping --------------------------------------------------------------#
print("Se inició el Web Scraping a 'Semana', por favor espere...") # Mensaje para visualización del inicio del proceso

# 8.2.1 Variable para definir el inicio de tiempo en ejecución
start = time.time()

Se inició el Web Scraping a 'Semana', por favor espere...


In [10]:
# PASO 8.3 Estructura base para acceder a la página en cuestión ----------------------------------------------------------------------#
service = Service(executable_path=r'../../chromedriver-win64/chromedriver.exe') # Creación del servicio mediante el driver
options = webdriver.ChromeOptions() # Definición de variable con las opciones para chrome
driver = webdriver.Chrome(service=service, options=options) # Creación del objeto del WebDriver

# Asignación de dataframe a una variable
links = IC.dateframe

# Control del error al momento de finalizar el páginado
try:
    
    # Iteración para acceso a los links del dataframe
    for i in range(len(links)):
        
        news = {
            "titles" : [],
            "summaries" : [],
            "authors" : [],
            "dates" : [],
            "paragraphs" : [],
            "themes" : [],
        }
        
        # Acceso a cada link del dataframe
        driver.get(links.iloc[i]['headers_link'])

        # Títulos
        # ****************************************************************************************************************
        title = driver.find_elements(By.XPATH, '//*[@id="fusion-app"]/div/main/article/div/div[2]/h1')
        if title:
            for i in title:
                print(titles.append(i.text))
        else: titles.append('...')
        # ****************************************************************************************************************

        # Resúmenes
        # ****************************************************************************************************************
        summary = driver.find_elements(By.XPATH, '//*[@id="fusion-app"]/div/main/article/div/div[2]/p')
        if summary: 
            for i in summary:
                print(summaries.append(i.text))
        else: summaries.append('...')
        # ****************************************************************************************************************

        # Autor
        # ****************************************************************************************************************
        date_a = driver.find_elements(By.XPATH, '//*[@id="fusion-app"]/div/main/article/div/div[2]/div[2]/div')
        if date_a: 
            if len(date_a) == 1:
                author_a = driver.find_elements(By.XPATH, '//*[@id="fusion-app"]/div/main/article/div/div[2]/div[2]/a/div')
                if author_a:
                    for i in author_a:
                        print(authors.append(i.text))
                else: authors.append('...')
            elif len(date_a) > 1:
                author_b = driver.find_elements(By.XPATH, '//*[@id="fusion-app"]/div/main/article/div/div[2]/div[2]/div[1]')
                if author_b:
                    for i in author_b:
                        print(authors.append(i.text))
                else: authors.append('...')
            else: authors.append('...')
        else: authors.append('...')
        # Test
        author_a = None
        author_b = None
        date_a = None
        # ****************************************************************************************************************

        # Fecha
        # ****************************************************************************************************************
        date_a = driver.find_elements(By.XPATH, '//*[@id="fusion-app"]/div/main/article/div/div[2]/div[2]/div')
        if date_a: 
            if len(date_a) == 1:
                if date_a:
                    for i in date_a:
                        print(dates.append(i.text))
                else: dates.append('...')
            elif len(date_a) > 1:
                date_b = driver.find_elements(By.XPATH, '//*[@id="fusion-app"]/div/main/article/div/div[2]/div[2]/div[2]')
                if date_b:
                    for i in date_b:
                        print(dates.append(i.text))
                else: dates.append('...')
            else: dates.append('...')
        else: dates.append('...')
        # Test
        author_a = None
        date_a = None
        date_b = None
        # ****************************************************************************************************************
        
        # Parrafos
        # ****************************************************************************************************************
        paragraphs_a = driver.find_elements(By.CLASS_NAME, 'prose')
        if paragraphs_a: 
            if len(paragraphs_a) > 1:
                paragraphs.append(JoinParagraphs(paragraphs_a))
            else:
                paragraphs.append(paragraphs_a[0].text)
        else: paragraphs.append('...')
        # ****************************************************************************************************************

        # Temas
        # ****************************************************************************************************************
        tags = driver.find_elements(By.CSS_SELECTOR, 'a.inline-block')
        if tags: 
            if len(tags) > 1:
                themes.append(JoinThemes(tags))
            else:
                themes.append(tags[0].text)
        else: themes.append('...')
        # ****************************************************************************************************************
                
        # Construcción del diccionario 
        news = {
            "titles" : titles,
            "summaries" : summaries,
            "authors" : authors,
            "dates" : dates,
            "paragraphs" : paragraphs,
            "themes" : themes
        }

        # Construcción inicial del dataframe
        df = pd.DataFrame(news)

        # (Opcional) Modo DEBUG, sirve para validar como el DataFrame se está llenando según las iteraciones
        display(df)
    else:
        driver.quit();
        
except Exception as e:
    display("Exception :", e)
    driver.quit() 
except NoSuchElementException as nse:
    display("NoSuchElementException :", nse)
    driver.quit()
except TimeoutException as tim:
    display("TimeoutException :", tim)
    driver.quit()

None
None
None
None


,titles,summaries,authors,dates,paragraphs,themes
0,África abre sus puertas a inversiones verdes e...,"El lunes, durante la inauguración de una histó...",Redacción Semana,4 de septiembre de 2023,África tiene una “oportunidad sin precedentes”...,"África, cambio climático, Clima, Kenia, Econom..."


None
None
None
None


,titles,summaries,authors,dates,paragraphs,themes
0,África abre sus puertas a inversiones verdes e...,"El lunes, durante la inauguración de una histó...",Redacción Semana,4 de septiembre de 2023,África tiene una “oportunidad sin precedentes”...,"África, cambio climático, Clima, Kenia, Econom..."
1,Inundaciones en Brasil: un testimonio impactan...,Las abundantes lluvias inundaron en pocas hora...,Redacción Semana,13 de septiembre de 2023,Los residentes de Muçum y de otras ciudades en...,"Brasil, Inundaciones, cambio climático, Lula d..."


None
None
None
None


...

,titles,summaries,authors,dates,paragraphs,themes
0,África abre sus puertas a inversiones verdes e...,"El lunes, durante la inauguración de una histó...",Redacción Semana,4 de septiembre de 2023,África tiene una “oportunidad sin precedentes”...,"África, cambio climático, Clima, Kenia, Econom..."
1,Inundaciones en Brasil: un testimonio impactan...,Las abundantes lluvias inundaron en pocas hora...,Redacción Semana,13 de septiembre de 2023,Los residentes de Muçum y de otras ciudades en...,"Brasil, Inundaciones, cambio climático, Lula d..."
2,Responsable climático de la ONU critica fuerte...,Las declaraciones de los países del G20 sobre ...,Redacción Semana,7 de septiembre de 2023,"El G20, un foro que aglutina a las principales...","G20, ONU, cambio climático, Gases de Efecto In..."
3,"“El cambio climático es real y es aterrador”, ...",SEMANA Habló con la galardonada directora Chai...,...,20 de junio de 2023,"Wild Life narra la vida de Kris Tompkins, una ...","Biodiversidad, Patagonia, National Geographic,..."
4,“Los últimos ocho años fueron los más caluroso...,"En 2022, el continente registró el verano más ...",...,20 de abril de 2023,El cambio climático golpeó a Europa con fuerza...,"calentamiento global, Medio ambiente, Europa,"
...,...,...,...,...,...,...
293,"Una Bogotá más verde, la meta del Jardín Botánico",La entidad planea sembrar 80.000 árboles y arb...,...,5 de agosto de 2021,Jardín Botánico de Bogotá\nÁrboles\nÁrboles de...,"Jardín Botánico de Bogotá, Árboles, Árboles de..."
294,La ONU y Los Andes promueven ideas de negocio ...,Lanzaron un concurso regional en el cual puede...,...,13 de julio de 2021,ONU\nUniversidad de los andes\nNegocios sosten...,"ONU, Universidad de los andes, Negocios sosten..."
295,"Océanos calientes, el combustible para los hur...",El huracán Iota tuvo un efecto devastador en P...,María Daniela Vargas/Pesquisa Javeriana,20 de noviembre de 2020,Océanos calientes por cambio climático\nCambio...,"Océanos calientes por cambio climático, Cambio..."
296,Crece la amenaza de conflictos por el agua en ...,El cambio climático y el rápido crecimiento de...,...,22 de marzo de 2020,Día Mundial del Agua\nConflictos por el agua\n...,"Día Mundial del Agua, Conflictos por el agua, ..."


None
None
None
None


,titles,summaries,authors,dates,paragraphs,themes
0,África abre sus puertas a inversiones verdes e...,"El lunes, durante la inauguración de una histó...",Redacción Semana,4 de septiembre de 2023,África tiene una “oportunidad sin precedentes”...,"África, cambio climático, Clima, Kenia, Econom..."
1,Inundaciones en Brasil: un testimonio impactan...,Las abundantes lluvias inundaron en pocas hora...,Redacción Semana,13 de septiembre de 2023,Los residentes de Muçum y de otras ciudades en...,"Brasil, Inundaciones, cambio climático, Lula d..."
2,Responsable climático de la ONU critica fuerte...,Las declaraciones de los países del G20 sobre ...,Redacción Semana,7 de septiembre de 2023,"El G20, un foro que aglutina a las principales...","G20, ONU, cambio climático, Gases de Efecto In..."
3,"“El cambio climático es real y es aterrador”, ...",SEMANA Habló con la galardonada directora Chai...,...,20 de junio de 2023,"Wild Life narra la vida de Kris Tompkins, una ...","Biodiversidad, Patagonia, National Geographic,..."
4,“Los últimos ocho años fueron los más caluroso...,"En 2022, el continente registró el verano más ...",...,20 de abril de 2023,El cambio climático golpeó a Europa con fuerza...,"calentamiento global, Medio ambiente, Europa,"
...,...,...,...,...,...,...
294,La ONU y Los Andes promueven ideas de negocio ...,Lanzaron un concurso regional en el cual puede...,...,13 de julio de 2021,ONU\nUniversidad de los andes\nNegocios sosten...,"ONU, Universidad de los andes, Negocios sosten..."
295,"Océanos calientes, el combustible para los hur...",El huracán Iota tuvo un efecto devastador en P...,María Daniela Vargas/Pesquisa Javeriana,20 de noviembre de 2020,Océanos calientes por cambio climático\nCambio...,"Océanos calientes por cambio climático, Cambio..."
296,Crece la amenaza de conflictos por el agua en ...,El cambio climático y el rápido crecimiento de...,...,22 de marzo de 2020,Día Mundial del Agua\nConflictos por el agua\n...,"Día Mundial del Agua, Conflictos por el agua, ..."
297,La mitad de las playas del mundo podrían desap...,"Un estudio, que profundiza en este efecto del ...",...,2 de marzo de 2020,Cambio climático\nPlayas de Colombia\nMedio am...,"Cambio climático, Playas de Colombia, Medio am..."


None
None
None
None


,titles,summaries,authors,dates,paragraphs,themes
0,África abre sus puertas a inversiones verdes e...,"El lunes, durante la inauguración de una histó...",Redacción Semana,4 de septiembre de 2023,África tiene una “oportunidad sin precedentes”...,"África, cambio climático, Clima, Kenia, Econom..."
1,Inundaciones en Brasil: un testimonio impactan...,Las abundantes lluvias inundaron en pocas hora...,Redacción Semana,13 de septiembre de 2023,Los residentes de Muçum y de otras ciudades en...,"Brasil, Inundaciones, cambio climático, Lula d..."
2,Responsable climático de la ONU critica fuerte...,Las declaraciones de los países del G20 sobre ...,Redacción Semana,7 de septiembre de 2023,"El G20, un foro que aglutina a las principales...","G20, ONU, cambio climático, Gases de Efecto In..."
3,"“El cambio climático es real y es aterrador”, ...",SEMANA Habló con la galardonada directora Chai...,...,20 de junio de 2023,"Wild Life narra la vida de Kris Tompkins, una ...","Biodiversidad, Patagonia, National Geographic,..."
4,“Los últimos ocho años fueron los más caluroso...,"En 2022, el continente registró el verano más ...",...,20 de abril de 2023,El cambio climático golpeó a Europa con fuerza...,"calentamiento global, Medio ambiente, Europa,"
...,...,...,...,...,...,...
295,"Océanos calientes, el combustible para los hur...",El huracán Iota tuvo un efecto devastador en P...,María Daniela Vargas/Pesquisa Javeriana,20 de noviembre de 2020,Océanos calientes por cambio climático\nCambio...,"Océanos calientes por cambio climático, Cambio..."
296,Crece la amenaza de conflictos por el agua en ...,El cambio climático y el rápido crecimiento de...,...,22 de marzo de 2020,Día Mundial del Agua\nConflictos por el agua\n...,"Día Mundial del Agua, Conflictos por el agua, ..."
297,La mitad de las playas del mundo podrían desap...,"Un estudio, que profundiza en este efecto del ...",...,2 de marzo de 2020,Cambio climático\nPlayas de Colombia\nMedio am...,"Cambio climático, Playas de Colombia, Medio am..."
298,Trancón de delfines rosados por bajo caudal en...,"Lo que sucede con esta especie, catalogada en ...",Antonio Paz/Mongabay Latam,18 de febrero de 2020,Delfines en la Orinoquía\nDelfines rosados baj...,"Delfines en la Orinoquía, Delfines rosados baj..."


In [11]:
df_news_clean = df.drop_duplicates(subset=["titles"])

In [12]:
df_news_clean.to_csv("df_news/df_news_clean_1.csv")

In [13]:
# Variable para definir el final de tiempo en ejecución
end = time.time()

# Variable para calcular tiempo y para enviar a la función
result = end-start

#Impresión del tiempo que tardó
print(Seconds(result))

El sitema tardó: 0 horas 25 minutos 18 segundos en finalizar su ejeción


In [14]:
df_news_clean

,titles,summaries,authors,dates,paragraphs,themes
0,África abre sus puertas a inversiones verdes e...,"El lunes, durante la inauguración de una histó...",Redacción Semana,4 de septiembre de 2023,África tiene una “oportunidad sin precedentes”...,"África, cambio climático, Clima, Kenia, Econom..."
1,Inundaciones en Brasil: un testimonio impactan...,Las abundantes lluvias inundaron en pocas hora...,Redacción Semana,13 de septiembre de 2023,Los residentes de Muçum y de otras ciudades en...,"Brasil, Inundaciones, cambio climático, Lula d..."
2,Responsable climático de la ONU critica fuerte...,Las declaraciones de los países del G20 sobre ...,Redacción Semana,7 de septiembre de 2023,"El G20, un foro que aglutina a las principales...","G20, ONU, cambio climático, Gases de Efecto In..."
3,"“El cambio climático es real y es aterrador”, ...",SEMANA Habló con la galardonada directora Chai...,...,20 de junio de 2023,"Wild Life narra la vida de Kris Tompkins, una ...","Biodiversidad, Patagonia, National Geographic,..."
4,“Los últimos ocho años fueron los más caluroso...,"En 2022, el continente registró el verano más ...",...,20 de abril de 2023,El cambio climático golpeó a Europa con fuerza...,"calentamiento global, Medio ambiente, Europa,"
...,...,...,...,...,...,...
295,"Océanos calientes, el combustible para los hur...",El huracán Iota tuvo un efecto devastador en P...,María Daniela Vargas/Pesquisa Javeriana,20 de noviembre de 2020,Océanos calientes por cambio climático\nCambio...,"Océanos calientes por cambio climático, Cambio..."
296,Crece la amenaza de conflictos por el agua en ...,El cambio climático y el rápido crecimiento de...,...,22 de marzo de 2020,Día Mundial del Agua\nConflictos por el agua\n...,"Día Mundial del Agua, Conflictos por el agua, ..."
297,La mitad de las playas del mundo podrían desap...,"Un estudio, que profundiza en este efecto del ...",...,2 de marzo de 2020,Cambio climático\nPlayas de Colombia\nMedio am...,"Cambio climático, Playas de Colombia, Medio am..."
298,Trancón de delfines rosados por bajo caudal en...,"Lo que sucede con esta especie, catalogada en ...",Antonio Paz/Mongabay Latam,18 de febrero de 2020,Delfines en la Orinoquía\nDelfines rosados baj...,"Delfines en la Orinoquía, Delfines rosados baj..."
